In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

# from selenium import webdriver
import re
from loguru import logger

/Users/divye/Desktop/MeakuChatBot/meaku/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
url0 = "https://help.hackerearth.com/"
url1 = "https://help.hackerearth.com/assessments-recruiters"

In [3]:
class get_HTML_tags(object):
    def __init__(self, url):
        self.url = url
        self.df = None

    def extract_links_and_text(self):
        """Extracts href links and corresponding text from a webpage and stores them in a pandas DataFrame.

        Args:
          url: The URL of the webpage to scrape.

        Returns:
          A pandas DataFrame containing the extracted href links and text.
        """
        # driver = webdriver.Firefox()
        page = requests.get(self.url)
        # html = driver.page_source
        soup = BeautifulSoup(page.text, "html")
        # driver.quit

        links_data = []
        for link in soup.find_all("a"):
            href = link.get("href")
            text = link.text.strip()
            links_data.append({"text": text, "href": href})

        self.df = pd.DataFrame(links_data)
        logger.debug("extracting urls is completed")

    def extract_headings_paragraphs(url, output_file):
        """Scrapes a webpage and saves the content to a text file.

        Args:
          url: The URL of the webpage to scrape.
          output_file: The path to the output text file.
        """

        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        text_content = ""
        for element in soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6", "p", "li"]):
            text_content += element.text + "\n"

        with open(output_file, "w", encoding="utf-8") as f:
            f.write(text_content)

    def is_valid_url(self, url):
        """Checks if the given URL is valid using a regular expression.

        Args:
          url: The URL to check.

        Returns:
          True if the URL is valid, False otherwise.
        """

        # Basic URL validation regex
        url_regex = r"https?://(?:www\\.)?[hackerearth./]?[ a-zA-Z0-9./]+"
        # url_regex = r"^(http|https)://?([\w.-]+(?:\.[\w.-]+)+)/?([\w.-]*)(/\w+(?:/\w+)*)*((\?\w+(=\w+)?(&\w+(=\w+)?)*)|(\#\w+)?)?$"
        return bool(re.match(url_regex, str(url)))

    def validate_urls(self):
        """Validates the 'href' column in a pandas DataFrame.

        Args:
          df: The pandas DataFrame containing the 'href' column.

        Returns:
          A new DataFrame with an additional column 'is_valid_url' indicating the validity of each URL.
        """
        if self.df.shape[0] == 0:
            return None

        self.df["is_valid_url"] = self.df["href"].apply(self.is_valid_url)
        self.df = self.df[self.df.is_valid_url == True].reset_index(drop=True)
        self.df.drop(columns=["is_valid_url"], inplace=True)
        logger.debug("Valid URLs are filtered")

In [4]:
all_urls = ["https://help.hackerearth.com/"]
df = pd.DataFrame({"home": all_urls[0]}, columns=["text", "href"])

for url in all_urls:
    get_tags = get_HTML_tags(url)
    get_tags.extract_links_and_text()
    get_tags.validate_urls()
    get_tags.df.drop_duplicates(keep="first", inplace=True)
    df = pd.concat([df, get_tags.df], axis=0)
    all_urls.pop(0)
    all_urls.extend(df.href.unique().tolist())

2024-08-09 18:42:49.227 | DEBUG    | __main__:extract_links_and_text:28 - extracting urls is completed
2024-08-09 18:42:49.229 | DEBUG    | __main__:validate_urls:60 - Valid URLs are filtered
2024-08-09 18:42:50.200 | DEBUG    | __main__:extract_links_and_text:28 - extracting urls is completed
2024-08-09 18:42:50.203 | DEBUG    | __main__:validate_urls:60 - Valid URLs are filtered
2024-08-09 18:42:51.776 | DEBUG    | __main__:extract_links_and_text:28 - extracting urls is completed
2024-08-09 18:42:51.779 | DEBUG    | __main__:validate_urls:60 - Valid URLs are filtered
2024-08-09 18:42:53.069 | DEBUG    | __main__:extract_links_and_text:28 - extracting urls is completed
2024-08-09 18:42:53.071 | DEBUG    | __main__:validate_urls:60 - Valid URLs are filtered
2024-08-09 18:42:54.649 | DEBUG    | __main__:extract_links_and_text:28 - extracting urls is completed
2024-08-09 18:42:54.658 | DEBUG    | __main__:validate_urls:60 - Valid URLs are filtered
2024-08-09 18:42:56.558 | DEBUG    | __m

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [5]:
df.shape

(12030, 2)

In [6]:
df.nunique()

text     980
href    1092
dtype: int64

In [8]:
df.to_csv("input_data/html_tags.csv", index=False)